# Chemical equilibrium with fixed volume and internal energy

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 7th, 2022</p>

Let's consider again the combustion of 1 mol of CH{{_4}} and 0.5 mol of O{{_2}}. This time, however, we will specify **volume and internal energy** as known properties at the equilibrium state.

We want to model the combustion of this gaseous mixture in a rigid and adiabatic chamber (volume and internal energy must be preserved). In this problem, both temperature and pressure are unknown and will be calculated together with the amounts of species that bring the system to a state of chemical equilibrium. 

Let's create our {{ChemicalSystem}} object (one with a single gaseous phase):

In [18]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

Next, we create an initial chemical state for this system:

In [19]:
state0 = ChemicalState(system)
state0.temperature(25.0, "celsius")
state0.pressure(1.0, "bar")
state0.set("CH4", 1.0, "mol")
state0.set("O2",  0.5, "mol")
props0 = ChemicalProps(state0)

V0 = props0.volume()  # the initial volume of the gases
U0 = props0.internalEnergy()  # the initial internal energy of the gases

print("INITIAL STATE")
print(state0)

INITIAL STATE
+-----------------+--------+------+
| Property        |  Value | Unit |
+-----------------+--------+------+
| Temperature     | 298.15 |    K |
| Pressure        | 100000 |   Pa |
| Charge:         |      0 |  mol |
| Element Amount: |        |      |
| :: H            |      4 |  mol |
| :: C            |      1 |  mol |
| :: O            |      1 |  mol |
| Species Amount: |        |      |
| :: CH4          |      1 |  mol |
| :: O2           |    0.5 |  mol |
| :: CO2          |  1e-16 |  mol |
| :: CO           |  1e-16 |  mol |
| :: H2O          |  1e-16 |  mol |
| :: H2           |  1e-16 |  mol |
+-----------------+--------+------+


Note above that we compute the chemical properties of the system in the initial state (using {{ChemicalProps}}) and store its initial volume and initial internal energy in the variables `V0` and `U0` respectively. 

Our chemical equilibrium problem imposes the following properties at equilibrium:

* volume; and
* internal energy.

The code below creates an {{EquilibriumSpecs}} object to reflect these specifications for the equilibrium constraints, which is then used to create our {{EquilibriumSolver}} object.

In [20]:
specs = EquilibriumSpecs(system)
specs.volume()
specs.internalEnergy()

solver = EquilibriumSolver(specs)

The next step is to create an {{EquilibriumConditions}} object in which we specify the values of volume and internal energy at equilibrium (which should be those values at the initial state). We do this in the next code block, which also sets lower bounds for temperature and pressure (to avoid negative values for these positive quantities during the equilibrium calculation!):

In [21]:
conditions = EquilibriumConditions(specs)
conditions.volume(V0)
conditions.internalEnergy(U0)

conditions.setLowerBoundTemperature(25.0, "celsius")
conditions.setLowerBoundPressure(1.0, "bar")

We have everything we need now to perform the equilibrium calculation, which is done next:

In [22]:
state = ChemicalState(state0)  # let's create a copy of the initial state

solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+-----------+------+
| Property        |     Value | Unit |
+-----------------+-----------+------+
| Temperature     |   1230.15 |    K |
| Pressure        |    768530 |   Pa |
| Charge:         |         0 |  mol |
| Element Amount: |           |      |
| :: H            |         4 |  mol |
| :: C            |         1 |  mol |
| :: O            |         1 |  mol |
| Species Amount: |           |      |
| :: CH4          |  0.102994 |  mol |
| :: O2           |     1e-16 |  mol |
| :: CO2          | 0.0261655 |  mol |
| :: CO           |   0.87084 |  mol |
| :: H2O          | 0.0768286 |  mol |
| :: H2           |   1.71718 |  mol |
+-----------------+-----------+------+


The printed state above does not show volume and internal energy. To verify our equilibrium state stored in `state` has volume and internal energy equal to `V0` and `U0` respectively, we do the following:

In [23]:
V = state.props().volume()
U = state.props().internalEnergy()

print("Volume at initial state:", V0, "m3")
print("Volume at final state:", V, "m3")

print("Internal energy at initial state:", U0, "J")
print("Internal energy at final state:", U, "J")

Volume at initial state: 0.0371844 m3
Volume at final state: 0.0371844 m3
Internal energy at initial state: -78318 J
Internal energy at final state: -78318 J


The verification above demonstrates we found an equilibrium state in which volume and internal energy were preserved.

Let's now check what is the final temperature and pressure when we burn 1 mol of CH{{_4}} and 0.5 mol of O{{_2}} in a rigid and adiabatic chamber:

In [25]:
T = units.convert(state.temperature(), "K", "degC")  # convert from K to °C
P = units.convert(state.pressure(), "Pa", "bar")  # convert from Pa to bar

print("Temperature at final state:", T, "°C")
print("Pressure at final state:", P, "bar")

Temperature at final state: 957.002 °C
Pressure at final state: 7.6853 bar


This concludes this tutorial, which demonstrated how Reaktoro can be used to perform chemical equilibrium calculations with given volume and internal energy.